In [2]:
import os
from pathlib import Path
from collections import defaultdict
from multiprocessing import Pool

import sys
sys.path.append('../src')
from command import run_cmd

In [7]:
ADAPTERS = '/home/chen1i6c04/Tools/shovill/db/trimmomatic.fa'
MIN_BQ = 3
TRIM_OPT = f"ILLUMINACLIP:{ADAPTERS}:2:30:10 CROP:300 HEADCROP:8 LEADING:{MIN_BQ} TRAILING:{MIN_BQ} SLIDINGWINDOW:4:20 MINLEN:36 TOPHRED33"


def trimming(forward_reads, reverse_reads, outdir, threads, prefix='trim'):
    forward_paired = os.path.join(outdir, f'{prefix}_R1.fq')
    reverse_paired = os.path.join(outdir, f'{prefix}_R2.fq')
    cmd = f"trimmomatic PE -threads {threads} {forward_reads} {reverse_reads} {forward_paired} /dev/null" \
          f" {reverse_paired} /dev/null {TRIM_OPT}"
    run_cmd(cmd)

In [8]:
trimming(
    '/media/NGS/MiSeqAnalysis/201030_M04855_0061_000000000-JCBFN/Data/Intensities/BaseCalls/R17-5974-TC2_S64_L001_R1_001.fastq.gz',
    '/media/NGS/MiSeqAnalysis/201030_M04855_0061_000000000-JCBFN/Data/Intensities/BaseCalls/R17-5974-TC2_S64_L001_R2_001.fastq.gz',
    '/home/chen1i6c04/test/3', 8)

In [4]:
dirpath = Path('/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/sub_reads')
outpath = Path('/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/trim_reads')

In [5]:
pairwise = defaultdict(list)

for i in dirpath.iterdir():
    prefix = i.stem.split('_', 1)[0]
    pairwise[prefix].append(i.as_posix())

In [6]:
pairwise

defaultdict(list,
            {'G6': ['/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/sub_reads/G6_R2.fq.gz',
              '/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/sub_reads/G6_R1.fq.gz'],
             'G3': ['/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/sub_reads/G3_R1.fq.gz',
              '/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/sub_reads/G3_R2.fq.gz'],
             'G0': ['/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/sub_reads/G0_R1.fq.gz',
              '/media/NGS/Data_Analysis/Aedes_aegypti_WGS/20200915/sub_reads/G0_R2.fq.gz']})

In [7]:
for prefix, seqfiles in pairwise.items():
    trimming(seqfiles[0], seqfiles[1], outpath, 8, prefix+'-trim')

In [19]:
if __name__=='__main__':
    p = Pool(8)
    try:
        for prefix, seqfiles in pairwise.items():
            p.apply_async(trimming, args=(seqfiles[0], seqfiles[1], outpath, 8, prefix))
#             p.apply_async(single_trimming, args=(seqfiles[0], outpath, 8, prefix))
        p.close()
        p.join()
    except:
        p.terminate()